# Sherlock Holmes Story Generator using HMMs

This notebook provides a walkthrough of Hidden Markov Modeling implementation for generating Sherlock Holmes stories.
We will use pure python and Natural Language ToolKit (NLTK) library to preprocess the data for building our Hidden Markov Model.

## Importing tools

These are the given libraries that we will be using:
* os: We use os library to create and specify the path for our dataset
* re: We use re to be able to use Regex functionality in python.
* nltk: The main library for interpreting our textual data. We will use it to tokeanize our text words
* random: We will use random to randomize our data generation
* glob: We use glob as a helper library to specify paths

In [49]:
import os
import re
import nltk
from nltk.tokenize import word_tokenize
import random
import glob

## Importing the SH dataset

The dataset that we use is called [Sherlock Holmes Stories](https://www.kaggle.com/datasets/idevji1/sherlock-holmes-stories) which is a collection of 67 files that contain Sherlock Holmes stories in a txt format

In [50]:
data_path = 'dataset/sherlock/'


def read_sh(data_path):
    txt = []
    file_paths = glob.glob(os.path.join(data_path, '*'))

    for file_path in file_paths:
        with open(file_path) as f:
            for line in f:
                line = line.strip()
                if line == '----------':
                    break
                if line:
                    txt.append(line)

    return txt


sherlock_stories = read_sh(data_path)
print(f"Number of lines: {len(sherlock_stories)}")

Number of lines: 215021


## Data Cleaning
To feed the data into our HMM model we need to clean it up. 
1. We convert all characters to lower
2. Remove special characters and replace it with '' (empty string) using regex
3. Convert the lines into tokens
4. Save only the tokens that are alphanumeric values

In [51]:
nltk.download('punkt')
def clean_data(data):
    cleaned_data = []
    for line in data:
        line = line.lower()
        line = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "",line)  # Replaces all the given characters with empty string ""
        tokens = word_tokenize(line)
        words = [word for word in tokens if word.isalpha()]
        cleaned_data+=words
    return cleaned_data

cleaned_stories=clean_data(sherlock_stories)
print(f"Number of words: {len(cleaned_stories)}")

[nltk_data] Downloading package punkt to C:\Users\Nifdi
[nltk_data]     Guliyev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Number of words: 2332110


## The HMM Model

**Prepare the Empty Model**: Start with an empty book where you'll jot down how words tend to follow each other.

**Read the Data**: Go through the cleaned data word by word (or element by element).

**Create State Pairs**: For each word, pick the current word and the word that comes after it. This pair is your "current state" and "next state". Imagine you're looking at two words at a time.

**Count Transitions**: Keep track of how many times you've seen each pair of words as you read through the data. This helps you understand which words tend to follow others.

**Calculate Probabilities**: After reading the data, go back to each pair of words you saw and calculate the chances of going from the current word to the next word. It's like figuring out how often each transition happens.

**Done!**: Your book now has information about which words often come after other words. This can help you make new sentences that sound like the original data.

Let's use a simple example to illustrate:

Suppose we have cleaned data: `['A', 'B', 'C', 'A', 'B', 'D']` and we're creating a Markov model with an n_gram of 2.

* You start with an empty book (dictionary).
* You read the data in pairs: `('A', 'B')`, `('B', 'C')`, `('C', 'A')`, `('A', 'B')`, `('B', 'D')`.
* You count the transitions: `('A', 'B')` appears twice, `('B', 'C')` appears once, `('C', 'A')` appears once, and so on.
* You calculate the probabilities: `('A', 'B')` has a 100% chance (2 out of 2 times), `('B', 'C')` has 100% chance (1 out of 1 time), and so on.
Your book now looks like:

`{'A': {'B': 1.0},
  'B': {'C': 1.0, 'D': 0.5, 'A': 0.5},
  'C': {'A': 1.0}}`
  
This means that if you start with 'A', you'll probably go to 'B'. If you start with 'B', you might go to 'C' or 'D'. If you start with 'C', you'll probably go to 'A'. This information helps you generate new sequences that are likely to sound similar to the original data.

In [52]:
def markov_model_builder(cleaned_data, n_gram=2):
    markov_model = {}
    for i in range(len(cleaned_data)-n_gram-1):
        current_state, next_state = '', ''
        for j in range(n_gram):
            current_state += cleaned_data[i+j] + ' '
            next_state += cleaned_data[i+j+n_gram] + ' '
        current_state  = current_state[:-1]
        next_state = next_state[:-1]
        
        if current_state not in markov_model:
            markov_model[current_state] = {}
            markov_model[current_state][next_state] = 1
        else:
            if next_state in markov_model[current_state]:
                markov_model[current_state][next_state] += 1
            else:
                markov_model[current_state][next_state] = 1
        
    for current_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[current_state][state] = count/total
    
    return markov_model

In [53]:
markov_model = markov_model_builder(cleaned_stories)

In [54]:
print(f"Number of states: {len(markov_model)}")

Number of states: 208670


In [55]:
print(f"All possible transitions from 'i am' state: \n {markov_model['i am']}")

All possible transitions from 'i am' state: 
 {'glad you': 0.004316546762589928, 'sorry one': 0.0008633093525179857, 'clearing out': 0.0008633093525179857, 'warned off': 0.0008633093525179857, 'going to': 0.007194244604316547, 'sure the': 0.002014388489208633, 'sure there': 0.0008633093525179857, 'not the': 0.0017266187050359713, 'ready to': 0.0057553956834532375, 'not so': 0.004028776978417266, 'sorry for': 0.0028776978417266188, 'obliged to': 0.0028776978417266188, 'afraid you': 0.0008633093525179857, 'here but': 0.0008633093525179857, 'wondering watson': 0.0008633093525179857, 'mr nathan': 0.0008633093525179857, 'not too': 0.0008633093525179857, 'unable to': 0.005179856115107914, 'not a': 0.010071942446043165, 'a bit': 0.0008633093525179857, 'none the': 0.0008633093525179857, 'about to': 0.006618705035971223, 'very busy': 0.002302158273381295, 'one of': 0.006618705035971223, 'convinced that': 0.01525179856115108, 'prepared to': 0.005179856115107914, 'afraid there': 0.001151079136690

## Generating SH Stories

**Get Ready**: Imagine you have a book (Markov model) that tells you which words come after other words. You want to use this book to write a new story.

**Start the Story**: Begin by writing a sentence based on a starting word. This word is your "current state".

**Write a New Sentence**: Keep writing more sentences based on the words that usually come after your current word. Pick the next word by looking at the chances (probabilities) given in your book.

**Repeat**: Keep writing more sentences until you've reached the desired story length.

**Finished!**: You now have a new story that sounds like the original data.

Let's use the same example and build upon it:

Suppose we have the Markov model built from before:

`{'A': {'B': 1.0},
 'B': {'C': 1.0, 'D': 0.5, 'A': 0.5},
 'C': {'A': 1.0}}`

And now, we're using the story_generator function to create a new story based on this model:

* You're getting ready to use your book (Markov model).
* You start your story with the sentence "my god".
* You look at your book for "my god" and see that the next word is usually "is". So, you write "my god is".
* You continue looking at your book. Since you've written "my god is", you check your book for "is" and see that "this" usually follows. So, you write "my god is this".
* You keep following your book and write the next words based on the chances given. If there's more than one option, you choose according to the probabilities.
* This continues until you've written enough sentences to reach the desired story length. You're using your Markov model to guide your writing, creating a new story that sounds like it could have come from the original data.

Here's an example of a possible generated story:

`my god is this a simple example hello world is this a simple example hello world is this a simple example hello world is this a simple example hello world is this a simple example hello world is this a simple example hello world is this a simple example hello world is this a simple example hello world is this a simple example
`

Please note that the generated story might vary each time you run the function due to the random choices made based on the probabilities.

In [68]:
def story_generator(markov_model, limit=150, start='my god'):
    n = 0
    current_state = start
    next_state = None
    story = ""
    story += current_state+" "
    while n<limit:
        next_state = random.choices(list(markov_model[current_state].keys()),
                                   list(markov_model[current_state].values()))
        current_state = next_state[0]
        story += current_state+" "
        n+=1
    return story

In [69]:
choice_list = ['my god', 'oh no', 'he has', 'no sir', 'it was', 'you made', 'the thing']

for i in range(20):
    print(f"Story {i}: {story_generator(markov_model, limit=8, start= random.choice(choice_list))}.")

Story 0: he has them hidden in a moment from his face was haggard and thin and eager stamped with .
Story 1: he has not been devoid of excitement when i say criminal because only a few hundred yards i .
Story 2: he has been called upon to interfere our investigation has served to show me the letter across it .
Story 3: oh no or to make myself as pitiable as possible i shall enumerate them to you in solving .
Story 4: the thing for which this man mcmurdo was a man answered by another hoot at a cabby we .
Story 5: it was clear to me then how is it then your brother held and that you saw all .
Story 6: he has amassed a fortune at last there is a bachelor years of age which is very rare .
Story 7: the thing which she had given an exclamation of satisfaction he bent forward and the paper upon the .
Story 8: the thing always appears to me by one the articles found his chubby face and his thin hands .
Story 9: the thing however is an old trick that you are close to him for some time but sir .

error: pathspec 'the' did not match any file(s) known to git
error: pathspec 'Markov' did not match any file(s) known to git
error: pathspec 'Model' did not match any file(s) known to git
error: pathspec 'for' did not match any file(s) known to git
error: pathspec 'story' did not match any file(s) known to git
error: pathspec 'generation'' did not match any file(s) known to git
